In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import pandas as pd
from tqdm import tqdm

In [5]:
root = "drive/MyDrive/data/bank/"

In [20]:
def reformat_csv(data):
  tmp = data.copy()
  tmp = tmp.fillna(0.0)
  tmp["Dátum"] = tmp["Dátum"].apply(lambda x: x[:4] + "-" + x[5:7] + "-" + x[8:])
  tmp["Árfolyam"] = tmp["Árfolyam"].apply(lambda x: float(str(x).replace(",", ".")))
  tmp["Napi befjegy. forgalom"] = tmp["Napi befjegy. forgalom"].apply(lambda x: float(str(x).replace(",", ".")))
  tmp["Napi befjegy. forgalom változás(%)"] = tmp["Napi befjegy. forgalom változás(%)"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["3 hónapos hozam"] = tmp["3 hónapos hozam"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["6 hónapos hozam"] = tmp["6 hónapos hozam"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["1 éves hozam"] = tmp["1 éves hozam"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["3 éves hozam"] = tmp["3 éves hozam"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["5 éves hozam"] = tmp["5 éves hozam"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["10 éves hozam"] = tmp["10 éves hozam"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  return tmp

In [63]:
with open(root + "data.json","r",encoding="utf8") as jf:
    stocks = json.load(jf)

In [65]:
for key in tqdm(stocks.keys()):
  stocks[key] = {
       "data" : pd.read_csv(root + "/raw/" + key + ".csv"),
       "name" : stocks[key]
  }

100%|██████████| 93/93 [00:01<00:00, 67.74it/s]


In [67]:
pbar = tqdm(stocks.keys())
for key in pbar:
  try:
    stocks[key]["reformated_data"] = reformat_csv(stocks[key]["data"])
    pbar.set_description(key)
  except:
    pbar.set_description("Errors - " + key)

HU0000709092: 100%|██████████| 93/93 [00:03<00:00, 30.31it/s]


In [68]:
[[key, stocks[key]["name"]] for key in stocks.keys() if 'reformated_data' not in stocks[key].keys()]

[['HU0000702881', 'OTP Paletta Nyíltvégű Értékpapír Alap'],
 ['HU0000702899', 'OTP Omega Fejlett Piaci Részvény Alapok Alapja A sorozat']]

In [72]:
pbar = tqdm(stocks.keys())
for key in pbar:
  pbar.set_description(key)
  if 'reformated_data' in stocks[key].keys():
    stocks[key]["reformated_data"].to_csv(root + "/cleaned/" + key + ".csv",
                                          sep=";",
                                          index=False,
                                          encoding='utf-8-sig',
                                          decimal=",")

HU0000709092: 100%|██████████| 93/93 [00:06<00:00, 15.03it/s]
